In [36]:
#Pkg.add("OffsetArrays") # -> Nos permitirá usar matrices que empiecen con índice 0 
using OffsetArrays

# Programación dinámica

Veremos únicamente casos que se calificarían como programación dinámica determinista en horizonte finito (categoría en la que caen la mayoría de problemas de optimización combinatoria determinista).

Utilizaremos dos ejemplos (knapsack y lot sizing) que ya hemos visto y que luego intentaremos combinar/comparar con los métodos expuestos con anterioridad. 

No voy a repasar los fundamentos de la programación dinámica. Si alguien necesita un repaso más allá del nivel introductorio, recomiendo el libro de Denardo porque es uno de los más accesibles y además es barato (E. Denardo, Dynamic Programming: Models and Applications).

# Uncapacitated Lot sizing

Este problema ya lo hemos visto en el diseño de cortes, así que hemos trasladado algunas funciones de la página anterior que nos resultarán útiles


In [28]:
function readULS(path::String)
    f = open(path,"r")
    T = parse(Int64,readline(f)) 
    c=map(x->parse(Int64,x),split(strip(readline(f))))
    h=map(x->parse(Int64,x),split(strip(readline(f))))
    K=map(x->parse(Int64,x),split(strip(readline(f))))
    d=map(x->parse(Int64,x),split(strip(readline(f))))
    close(f)
    T, c, h, K, d
end
# T -> número de periodos
# c_t -> costo de comprar una unidad en el periodo t
# h_t -> costo de inventariar una unidad en el periodo t
# K_t -> costo de realizar un pedido en el periodo t
# d_t -> demanda en el periodo t

readULS (generic function with 1 method)

In [29]:
#run(`wget -O test.dat https://raw.githubusercontent.com/sbebo/julia-cuts/master/test.dat`)
#run(`cat test.dat`)
T, c, h, K, d = readULS("test.dat")

(200,[1,4,1,2,4,4,1,4,1,3  …  2,1,1,1,4,3,3,3,3,2],[2,3,2,2,3,2,3,2,1,1  …  3,2,2,1,2,3,1,1,2,1],[10,8,6,6,10,8,9,5,10,10  …  7,9,6,6,7,7,7,5,7,10],[4,10,5,4,9,7,2,8,4,7  …  9,4,2,2,10,10,4,7,8,3])

## Uncapacitated Lot Sizing. Método 1. Estados asociados al nivel de inventario y recurrencia backward

Éste sería el método tradicional para empezar en clase. Sabemos que en el periodo T+1 el costo es 0 y la última decisión corresponde al periodo T. 

In [43]:
#Pkg.add("OffsetArrays")

function DP_m1(T,c,h,K,d)
    D=sum(d)
    tableCosts = OffsetArray(Int64, 1:T, 0:D)
    tablePath = OffsetArray(Int64, 1:T, 0:D)
    for i=0:d[T]-1
        tableCosts[1,i]=K[T]+c[T]*(d[T]-i) #hacer un pedido y comprar lo que falta hasta d[T]
    end
    for i=d[T]:D
        tableCosts[1,i]=K[T]+h[T]*(i-d[T]) #hacer un pedido y comprar lo que falta hasta d[T]
    end
    for t=2:T #para cada etapa
        period=T-t+1
        for st=0:D
            #hay que diferenciar dos casos
            if st<d[period] #es obligatorio comprar
                #iniciamos con la opción de comprar justo para la semana
                tableCosts[t,st]=K[period]+c[period]*(d[period]-st)+tableCosts[t-1,0]
                tablePath[t,st]=0
                #resto de opciones consisten en comprar desde d[period]-st+1 hasta D-st
                for tr=d[period]-st+1:D-st
                    cost=K[period]+c[period]*tr+h[period]*(st+tr-d[period])+tableCosts[t-1,st+tr-d[period]]
                    if cost < tableCosts[t,st]
                        tableCosts[t,st]=cost
                        tablePath[t,st]=st+tr-d[period]
                    end
                end
            else
                #inicializamos con la opción de no comprar
                tableCosts[t,st]=h[period]*(st-d[period])+tableCosts[t-1,st-d[period]]
                tablePath[t,st]=st-d[period]
                #resto de opciones consisten en comprar desde 1 hasta D-st
                for tr=1:D-st
                    cost=K[period]+c[period]*tr+h[period]*(st+tr-d[period])+tableCosts[t-1,st+tr-d[period]]
                    if cost<tableCosts[t,st]
                        tableCosts[t,st]=cost
                        tablePath[t,st]=st+tr-d[period]
                    end
                end
            end
        end
    end
    println(tableCosts[1,:])
    println("\n\n")
    println(tableCosts[2,:])
    println("\n\n")
    println(tableCosts[3,:])
    println("\n\n")
    println(tableCosts[4,:])
    println("\n\n")
    println(tableCosts[5,:])

end
T=5
c=[10 10 10 10 10]
h=[1 1 1 1 1 ]
K=[100 100 100 100 100]
d=[5 6 7 12 8]
DP_m1(T,c,h,K,d)


[180,170,160,150,140,130,120,110,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130]



[400,390,380,370,360,350,340,330,320,310,300,290,180,171,162,153,144,135,126,117,108,110,112,114,116,118,120,122,124,126,128,130,132,134,136,138,140,142,144]



[482,472,462,452,442,432,422,400,391,382,373,364,355,346,337,328,319,310,301,192,184,176,168,160,152,144,136,128,131,134,137,140,143,146,149,152,155,158,161]



[561,551,541,531,521,511,482,473,464,455,446,437,428,407,399,391,383,375,367,359,351,343,335,327,319,211,204,197,190,183,176,169,162,155,159,163,167,171,175]



[636,626,616,606,596,561,552,543,534,525,516,488,480,472,464,456,448,440,420,413,406,399,392,385,378,371,364,357,350,343,236,230,224,218,212,206,200,194,188]


## Uncapacitated Lot Sizing. Método 2. Estados asociados al nivel de inventario y recurrencia forward


## Uncapacitated Lot Sizing. Método 3. Estados asociados al nivel de inventario, recurrencia forward y transiciones limitadas 

## Uncapacitated Lot Sizing. Método 4. Estados asociados a los periodos y recurrencia forward
